In [8]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

In [9]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

# Ensure the directory for output exists
output_dir = '~/HW2/eda_results/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Load the cleaned dataset
data = pd.read_csv('~/data/heart_disease_cleaned.csv')

# Calculate and save the correlation matrix
correlation_matrix = data.corr()
correlation_matrix.to_csv(f'{output_dir}/correlation_matrix.csv')

# Visualize the correlation matrix
plt.figure(figsize=(15, 15))
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap='coolwarm', square=True)
plt.title('Correlation Matrix')
plt.savefig(f'{output_dir}/correlation_matrix.png')
plt.close()

# Extract high correlations and suggest potential interaction terms
high_corr = correlation_matrix.abs().unstack()
high_corr = high_corr[high_corr != 1]  # Remove self-correlation
high_corr = high_corr.drop_duplicates()  # Avoid duplication in correlation pairs
high_corr = high_corr[high_corr > 0.7]  # Filter very high correlations

# Save high correlations
high_corr.sort_values(ascending=False).to_csv(f'{output_dir}/high_correlations.csv')

# Store potential interaction terms based on high correlations
with open(f'{output_dir}/potential_interaction_terms.txt', 'w') as f:
    f.write("Potential interaction terms based on high correlations:\n")
    for (var1, var2), corr in high_corr.items():
        if corr > 0.7:
            f.write(f"{var1} * {var2} (Correlation: {corr:.2f})\n")

# Calculate skewness and kurtosis for each variable
skew_kurt = data.agg(['skew', 'kurt']).transpose()
skew_kurt.columns = ['Skewness', 'Kurtosis']
skew_kurt.to_csv(f'{output_dir}/skewness_kurtosis.csv')

# Suggest transformations for skewed data
transform_suggestions = {}
for column, stats in skew_kurt.iterrows():
    if abs(stats['Skewness']) > 1:
        suggestion = 'Consider log transformation' if stats['Skewness'] > 0 else 'Consider square root transformation'
        transform_suggestions[column] = suggestion

# Save transformation suggestions
with open(f'{output_dir}/transformation_suggestions.txt', 'w') as f:
    for column, suggestion in transform_suggestions.items():
        f.write(f"{column}: {suggestion}\n")
        
# Plot distributions of all variables
for column in data.columns:
    plt.figure()
    sns.histplot(data[column], kde=True)
    plt.title(f'Distribution of {column}')
    plt.xlabel(column)
    plt.ylabel('Frequency')
    plt.savefig(f'{output_dir}/distribution_{column}.png')
    plt.close()
